In [20]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.svm import SVR

In [9]:
df=pd.read_csv('sp500-m15.csv')
df

,row,date,open,high,low,close,tick_vol,vol,spread,UP,DN,ATR_LH,ATR_LO,ARR_UP,ARR_DN
0,0,2024.01.16 14:30:00,4763.93,4769.68,4761.93,4768.68,346,346,260,4758.43,0.0,4.753042e+03,4.763818e+03,0.0000,0.0
1,1,2024.01.16 14:15:00,4763.93,4764.93,4758.43,4762.18,383,383,260,4758.43,0.0,4.753062e+03,4.763798e+03,0.0000,0.0
2,2,2024.01.16 14:00:00,4768.43,4768.93,4763.43,4763.68,258,258,260,4757.68,0.0,4.752322e+03,4.763038e+03,0.0000,0.0
3,3,2024.01.16 13:45:00,4766.43,4769.43,4765.93,4768.18,242,242,260,4757.68,0.0,4.752274e+03,4.763086e+03,0.0000,0.0
4,4,2024.01.16 13:30:00,4763.43,4768.18,4761.93,4766.18,311,311,260,4757.68,0.0,4.752284e+03,4.763076e+03,4752.2842,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100951,100951,1970.01.01 00:00:00,0.00,0.00,0.00,0.00,0,0,0,0.00,0.0,1.797693e+308,-1.797693e+308,0.0000,0.0
100952,100952,1970.01.01 00:00:00,0.00,0.00,0.00,0.00,0,0,0,0.00,0.0,1.797693e+308,-1.797693e+308,0.0000,0.0
100953,100953,1970.01.01 00:00:00,0.00,0.00,0.00,0.00,0,0,0,0.00,0.0,1.797693e+308,-1.797693e+308,0.0000,0.0
100954,100954,1970.01.01 00:00:00,0.00,0.00,0.00,0.00,0,0,0,0.00,0.0,1.797693e+308,-1.797693e+308,0.0000,0.0


In [10]:
df.date=pd.to_datetime(df.date)
df.sort_values('date', ascending=True,inplace=True)
df.reset_index(inplace=True,drop=True)

In [11]:
df['diff-vols']=df.tick_vol - df.vol
df['diff-vols'].value_counts()

diff-vols
0    100956
Name: count, dtype: int64

In [12]:
to_drop=['tick_vol','diff-vols']
df.drop(columns=to_drop,inplace=True)
df

,row,date,open,high,low,close,vol,spread,UP,DN,ATR_LH,ATR_LO,ARR_UP,ARR_DN
0,100955,1970-01-01 00:00:00,0.00,0.00,0.00,0.00,0,0,0.00,0.0,1.797693e+308,-1.797693e+308,0.0000,0.0
1,100324,1970-01-01 00:00:00,0.00,0.00,0.00,0.00,0,0,0.00,0.0,3.213200e+00,-3.213200e+00,0.0000,0.0
2,100323,1970-01-01 00:00:00,0.00,0.00,0.00,0.00,0,0,0.00,0.0,3.223200e+00,-3.223200e+00,0.0000,0.0
3,100322,1970-01-01 00:00:00,0.00,0.00,0.00,0.00,0,0,0.00,0.0,3.230700e+00,-3.230700e+00,0.0000,0.0
4,100321,1970-01-01 00:00:00,0.00,0.00,0.00,0.00,0,0,0.00,0.0,3.245700e+00,-3.245700e+00,0.0000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100951,4,2024-01-16 13:30:00,4763.43,4768.18,4761.93,4766.18,311,260,4757.68,0.0,4.752284e+03,4.763076e+03,4752.2842,0.0
100952,3,2024-01-16 13:45:00,4766.43,4769.43,4765.93,4768.18,242,260,4757.68,0.0,4.752274e+03,4.763086e+03,0.0000,0.0
100953,2,2024-01-16 14:00:00,4768.43,4768.93,4763.43,4763.68,258,260,4757.68,0.0,4.752322e+03,4.763038e+03,0.0000,0.0
100954,1,2024-01-16 14:15:00,4763.93,4764.93,4758.43,4762.18,383,260,4758.43,0.0,4.753062e+03,4.763798e+03,0.0000,0.0


In [13]:
df = df[(df.close != 0.0)].reset_index(drop=True)
df.shape

(100000, 14)

In [14]:
df = df.groupby(df['date'].dt.date).agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'vol': 'sum',
    'ATR_LH': 'mean',
    'ATR_LO': 'last',
    'ARR_UP': 'sum',
    'ARR_DN': 'sum',
    'spread': 'mean',
    'UP': 'sum',
    'DN': 'sum'
}).reset_index()

In [16]:
df['date']=pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

In [17]:
df=df.drop(columns=['date'])

In [18]:
df['target'] = df['close'].shift(-1)
df = df.dropna(subset=['target'])

In [21]:
split_point = int(len(df) * 0.8)
train_df = df[:split_point].copy()
test_df = df[split_point:].copy()
X_train = train_df.drop(columns=['target'])
y_train = train_df['target']
X_test = test_df.drop(columns=['target'])
y_test = test_df['target']
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train_sc = sc_X.fit_transform(X_train)
X_test_sc = sc_X.transform(X_test)
y_train_sc = sc_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_sc = sc_y.transform(y_test.values.reshape(-1, 1))
svr = SVR()
svr.fit(X_train_sc, y_train_sc.ravel())
y_pred_train = svr.predict(X_train_sc)
y_pred_test = svr.predict(X_test_sc)

r2_score(y_train_sc, y_pred_train),r2_score(y_test_sc, y_pred_test)

(0.9940276951069847, 0.6520121126331102)